In [23]:
import cv2
import numpy as np

# open input video
cap = cv2.VideoCapture('10sec.mp4')

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

bufCap = 37

# record the video
out1 = cv2.VideoWriter('original.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
out2 = cv2.VideoWriter('10secMEDIAN.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

buf = np.zeros((bufCap, frame_height, frame_width, 3),np.uint8)
bg = np.zeros((frame_height, frame_width, 3),np.uint8)

while(1):
    # read the frame
    ret, frame = cap.read()
    
    if ret is True:
        
        # convert to greyscale
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # add frame to buffer
        buf[0] = frame
        
#         cv2.imshow('frame0', buf[0])
        
        # rotate buffer by 1 position to move the latest frame to the end of the buffer
        buf = np.roll(buf, 1, axis=0)
#         print(np.shape(buf))
        bg = np.median(buf, axis=0, out=bg)
        bg = cv2.cvtColor(bg, cv2.COLOR_GRAY2BGR)
        
        # write original frame
#         out1.write(frame)
        # write calculated background frame
        out2.write(bg)
        
        cv2.imshow('bg', bg)
        
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
    
cap.release()
out1.release()
out2.release()

cv2.destroyAllWindows()

ValueError: could not broadcast input array from shape (480,640) into shape (480,640,3)

In [25]:
import cv2
import numpy as np

def mode(ndarray, axis=0):
    # Check inputs
    ndarray = np.asarray(ndarray)
    ndim = ndarray.ndim
    if ndarray.size == 1:
        return (ndarray[0], 1)
    elif ndarray.size == 0:
        raise Exception('Cannot compute mode on empty array')
    try:
        axis = range(ndarray.ndim)[axis]
    except:
        raise Exception('Axis "{}" incompatible with the {}-dimension array'.format(axis, ndim))

    # If array is 1-D and np version is > 1.9 np.unique will suffice
    if all([ndim == 1,
            int(np.__version__.split('.')[0]) >= 1,
            int(np.__version__.split('.')[1]) >= 9]):
        modals, counts = np.unique(ndarray, return_counts=True)
        index = np.argmax(counts)
        return modals[index], counts[index]

    # Sort array
    sort = np.sort(ndarray, axis=axis)
    # Create array to transpose along the axis and get padding shape
    transpose = np.roll(np.arange(ndim)[::-1], axis)
    shape = list(sort.shape)
    shape[axis] = 1
    # Create a boolean array along strides of unique values
    strides = np.concatenate([np.zeros(shape=shape, dtype='bool'),
                                 np.diff(sort, axis=axis) == 0,
                                 np.zeros(shape=shape, dtype='bool')],
                                axis=axis).transpose(transpose).ravel()
    # Count the stride lengths
    counts = np.cumsum(strides)
    counts[~strides] = np.concatenate([[0], np.diff(counts[~strides])])
    counts[strides] = 0
    # Get shape of padded counts and slice to return to the original shape
    shape = np.array(sort.shape)
    shape[axis] += 1
    shape = shape[transpose]
    slices = [slice(None)] * ndim
    slices[axis] = slice(1, None)
    # Reshape and compute final counts
    counts = counts.reshape(shape).transpose(transpose)[slices] + 1

    # Find maximum counts and return modals/counts
    slices = [slice(None, i) for i in sort.shape]
    del slices[axis]
    index = np.ogrid[slices]
    index.insert(axis, np.argmax(counts, axis=axis))
    return sort[index], counts[index]



# open input video
cap = cv2.VideoCapture(0)

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

print(frame_width)
print(frame_height)

bufCap = 21

# record the video
out1 = cv2.VideoWriter('original.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
out2 = cv2.VideoWriter('10secMODE.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

buf = np.zeros((bufCap, frame_height, frame_width),np.uint8)
bg = np.zeros((frame_height, frame_width),np.uint8)

while(1):
    # read the frame
    ret, frame = cap.read()
    
    if ret is True:
        
        # convert to greyscale
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # add frame to buffer
        buf[0] = frame
        
        cv2.imshow('frame0', buf[0])
        
        # rotate buffer by 1 position to move the latest frame to the end of the buffer
        buf = np.roll(buf, 1, axis=0)
        
#         bg = np.median(buf, axis=0, out=bg)
#         unique, counts = np.unique(buf, axis=0, return_counts = True)
#         index = np.argmax(counts, axis=1)
#         print(np.shape(buf))
#         print(np.shape(unique))
#         print(np.shape(counts))
#         print(np.shape(index))
        bg, counts = mode(buf, axis = 0)
#         print(bg)
#         break
#         bg = unique[index]
#         print(np.shape(bg))
        
        # write original frame
#         out1.write(frame)
        # write calculated background frame
        print(np.shape(bg))
#         bg = bg.T
#         bg = cv2.resize(bg, (1280, 720*3))
        bg = cv2.cvtColor(bg, cv2.COLOR_GRAY2BGR)
        
        print(np.shape(bg))
        
        out2.write(bg)
        
        cv2.imshow('bg', bg)
        
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
    
cap.release()
out1.release()
out2.release()

cv2.destroyAllWindows()

640
480
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 

(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)
(480, 640)
(480, 640, 3)


In [33]:
print(counts)
print(unique)

[36  1]
[[[  0   0   0 ...,   0   0   0]
  [  0   0   0 ...,   0   0   0]
  [  0   0   0 ...,   0   0   0]
  ..., 
  [  0   0   0 ...,   0   0   0]
  [  0   0   0 ...,   0   0   0]
  [  0   0   0 ...,   0   0   0]]

 [[253 253 253 ...,  57  65  74]
  [253 253 253 ...,  60  69  75]
  [253 253 253 ...,  65  67  80]
  ..., 
  [ 44  45  46 ...,  97  97  99]
  [ 45  44  45 ...,  98 103 101]
  [ 45  44  45 ..., 100 103 101]]]
